# LangChain Master Class
I will be using this notebook to take notes on the [LangChain Master Class For Beginners 2024](https://www.youtube.com/watch?v=yF9kGESAi3M) video. LangChain is a very good starting point for getting in touch with OpenAI's API and is significantly easier to use. This tutorial starts from basic features like interacting with an LLM or designing a real time conversation with it and progressively works its way towards more complex features, like chaining and ReAct agents. Let's start with the basics.

## 1. Interacting with chat models
Here, we learn the library's basics, like interacting with models like ChatGPT.

### Chat model basics

In [ ]:
from langchain_openai import ChatOpenAI

# Create the model
model = ChatOpenAI(model="gpt-4o-mini")

# Invoke the model with a message
result = model.invoke("What is 81 divided by 9?")
print("Full result:")
print(result)
print("Content only:")
print(result.content)

### Basic Conversation

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Create the model
model = ChatOpenAI(model="gpt-4o-mini")

# SystemMessage: Message to manage AI behavior
# HumanMessage: Message from a human to the AI model
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
]

# Invoke the model with a message
result = model.invoke(messages)
print(f"ChatGPT-4o-mini: {result.content}")

# # AIMessage: Response from an AI
# messages = [
#     SystemMessage(content="Solve the following math problems"),
#     HumanMessage(content="What is 81 divided by 9?"),
#     AIMessage(content="81 divided by 9 is 9."),
#     HumanMessage(content="What is 10 times 5?"),
# ]
# # Invoke the model with a message
# result = model.invoke(messages)
# print(f"ChatGPT-4o-mini: {result.content}")

### Real time conversation

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Create the model
model = ChatOpenAI(model="gpt-4o-mini")

chat_history = []

system_mesage = SystemMessage(content="You are a helpful AI assistant that answers questions in 2-3 sentences.")
chat_history.append(system_mesage)

while True:
    # Read user input
    query = input("You: ")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))
    print(f"ChatGPT-4o-mini: {response}")

print("----- Message History -----")
for message in chat_history:
    print(message)

## 2. Prompt templates
Now that we got the hang of interacting with a chat model, let's understand the basics of prompt templates and how to use them effectively.

### Basic templates

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

# # Template using a template string
# template = "Tell me a joke about {topic}."
# prompt_template = ChatPromptTemplate.from_template(template)

# print("-----Prompt from Template-----")
# prompt = prompt_template.invoke({"topic": "cats"})
# print(prompt)



# # Template using multiple placeholders
# template = """You are a helpful assistant.
# Human: Tell me a {adjective} story about a {animal}.
# Assistant:"""
# prompt_template = ChatPromptTemplate.from_template(template)
# prompt = prompt_template.invoke({"adjective": "funny", "animal": "panda"})
# print("-----Prompt from Template-----")
# print(prompt)



# System and human messages
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes"),
]

prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
print("-----Prompt from Template-----")
print(prompt)



# #! THIS DOES NOT WORK!
# messages = [
#     ("system", "You are a comedian who tells jokes about {topic}."),
#     HumanMessage(content="Tell me {joke_count} jokes"),
# ]

# prompt_template = ChatPromptTemplate.from_messages(messages)
# prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
# print("-----Prompt from Template-----")
# print(prompt)

### Prompt template with chat model

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Create the model
model = ChatOpenAI(model="gpt-4o-mini")

# Template using a template string
print("-----Prompt from template-----")
template = "Tell me a joke about {topic}."
prompt_template = ChatPromptTemplate.from_template(template)
prompt = prompt_template.invoke({"topic": "cats"})
result = model.invoke(prompt)
response = result.content
print(f"ChatGPT-4o-mini: {response}")


# Template using multiple placeholders
print("-----Prompt from template with multiple placeholders-----")
template = """You are a helpful assistant.
Human: Tell me a {adjective} story about a {animal}.
Assistant:"""
prompt_template = ChatPromptTemplate.from_template(template)
prompt = prompt_template.invoke({"adjective": "funny", "animal": "panda"})
result = model.invoke(prompt)
response = result.content
print(f"ChatGPT-4o-mini: {response}")

# Template with system and human messages
print("-----Prompt from template with system and human messages-----")
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes"),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
result = model.invoke(prompt)
response = result.content
print(f"ChatGPT-4o-mini: {response}")


## 3. Chains
Now that we are familiar with chat models and prompts, it is time to build chains to automate tasks!

### Chain basics

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

# Create the model
model = ChatOpenAI(model="gpt-4o-mini")

# Define prompt templates
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes"),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

# Create a chain and make output readable
chain = prompt_template | model | StrOutputParser()

# Run the chain
result = chain.invoke({"topic": "dogs", "joke_count": 3})
print(result)

# Chain parallels

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI

# Create the model
model = ChatOpenAI(model="gpt-4o-mini")

# Define prompt templates
messages = [
    ("system", "You are an expert product reviewer."),
    ("human", "List me the main features of the product {product_name}"),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

def analyze_pros(features):
    messages = [
        ("system", "You are an expert product reviewer."),
        ("human", "Given these features: {features}, list the pros of these features."),
    ]
    pros_template = ChatPromptTemplate.from_messages(messages)
    return pros_template.format_prompt(features=features)

# Pros branch
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

def analyze_cons(features):
    messages = [
        ("system", "You are an expert product reviewer."),
        ("human", "Given these features: {features}, list the cons of these features."),
    ]
    cons_template = ChatPromptTemplate.from_messages(messages)
    return cons_template.format_prompt(features=features)

# Cons branch
cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

# Create a chain and make output readable
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

# Run the chain
result = chain.invoke({"product_name": "The Latest iPhone"})
print(result)

### Chain branches

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_openai import ChatOpenAI

# Create the model
model = ChatOpenAI(model="gpt-4o-mini")

# Define prompt templates
positive_messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Generate a thank you note for this positive feedback: {feedback}"),
]
positive_feedback_template = ChatPromptTemplate.from_messages(positive_messages)

negative_messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Generate a response addressing this negative feedback: {feedback}"),
]
negative_feedback_template = ChatPromptTemplate.from_messages(negative_messages)

neutral_messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Generate a request for more details for this neutral feedback: {feedback}"),
]
neutral_feedback_template = ChatPromptTemplate.from_messages(neutral_messages)

escalate_messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Generate a message to escalate this feedback to a human agent: {feedback}"),
]
escalate_feedback_template = ChatPromptTemplate.from_messages(escalate_messages)

# Define a feedback classification template
classification_messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}"),
]
classification_template = ChatPromptTemplate.from_messages(classification_messages)

# Define the runnable branches
branches = RunnableBranch(
    (lambda x: "positive" in x, positive_feedback_template | model | StrOutputParser()),
    (lambda x: "negative" in x, negative_feedback_template | model | StrOutputParser()),
    (lambda x: "neutral" in x, neutral_feedback_template | model | StrOutputParser()),
    escalate_feedback_template | model | StrOutputParser()
)

# Create a chain which branches according to the LLM's classification
chain = classification_template | model | StrOutputParser() | branches

# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"
print ("-----Good review-----")
review = "The product is excellent. I really enjoyed using it and found it very helpful."
result = chain.invoke({"feedback": review})
print(result)

print ("-----Bad review-----")
review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})
print(result)

print ("-----Neutral review-----")
review = "The product is okay. It works as expected but nothing exceptional."
result = chain.invoke({"feedback": review})
print(result)

print ("-----Default-----")
review = "I'm not sure about the product yet. Can you tell me more about its features and benefits?"
result = chain.invoke({"feedback": review})
print(result)

## 4. Agents & Tools
Finally, let's learn about agents, how they work, and how to build custom tools to enhance their capabilities.

### Agent basics

In [ ]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

# This function will be used as a tool that returns the current time
def get_current_time(*args, **kwargs):
    """Returns the current military time in HH:MM:SS format."""
    from datetime import datetime
    return datetime.now().strftime("%H:%M:%S")

# List of tools available to the agent
tools = []
current_time_tool = Tool(
    name="Time",
    func=get_current_time,
    description="Useful for when you need to know the current time"
)
tools.append(current_time_tool)

# Pull the prompt template from the hub. This prompt follows ReAct = Reason and Action
# https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")

# Create the model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create a ReAct agent
agent = create_react_agent(llm=model, tools=tools, prompt=prompt, stop_sequence=True)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Run the agent
response = agent_executor.invoke({"input" : "What time is it?"})
print(f"ChatGPT-4o-mini: {response['output']}")


### ReAct agent chat

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI


# Define Tools
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime

    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")


def search_wikipedia(query):
    """Searches Wikipedia and returns the summary of the first result."""
    from wikipedia import summary

    try:
        # Limit to two sentences for brevity
        return summary(query, sentences=2)
    except:
        return "I couldn't find any information on that."


# Define the tools that the agent can use
tools = [
    Tool(
        name="Time",
        func=get_current_time,
        description="Useful for when you need to know the current time.",
    ),
    Tool(
        name="Wikipedia",
        func=search_wikipedia,
        description="Useful for when you need to know information about a topic.",
    ),
]

# Load the correct JSON Chat Prompt from the hub
prompt = hub.pull("hwchase17/structured-chat-agent")

# Initialize a ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o")

# Create a structured Chat Agent with Conversation Buffer Memory
# ConversationBufferMemory stores the conversation history, allowing the agent to maintain context across interactions
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)

# create_structured_chat_agent initializes a chat agent designed to interact using a structured prompt and tools
# It combines the language model (llm), tools, and prompt to create an interactive agent
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

# AgentExecutor is responsible for managing the interaction between the user input, the agent, and the tools
# It also handles memory to ensure context is maintained throughout the conversation
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

# Initial system message to set the context for the chat
# SystemMessage is used to define a message from the system to the agent, setting initial instructions or context
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Time and Wikipedia."
memory.chat_memory.add_message(SystemMessage(content=initial_message))

# Chat Loop to interact with the user
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))